Here we will work on the challenge faced in previous RAG application :

-  Missing of traces for run_name of custom python functions


With the help of `@traceable` we can trace those custom python functions.

In [4]:
# Libraries Installation
# pip install -qU langchain_google_genai langchain langchain-community faiss-cpu pypdf langsmith

# Authenticate User
from google.colab import auth
auth.authenticate_user()

import os
from google.colab import userdata

# Configure Environment Variables
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "RAG_Chatbot"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")

# Import libraries
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tracers.langchain import LangChainTracer
from langsmith import traceable
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda


# DEFINE MODEL
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    project=userdata.get("GOOGLE_CLOUD_PROJECT"),
    location="global",
    temperature=0,
    vertexai=True
)

# Create a tracer for the specific project
tracer = LangChainTracer(project_name="RAG_Chatbot")

# Create a custom configurations
custom_configurations = {
    "callbacks": [tracer],
    "run_name" : "pdf_rag_query"
}

# Locate the PDF
PDF_PATH = "/content/islr.pdf"

# ---------- traced setup steps ----------
# We explicitly pass project_name to ensure it uses the correct project even if env vars are sticky
@traceable(name="load_pdf", project_name="RAG_Chatbot")
def load_pdf(path: str):
    loader = PyPDFLoader(path)
    return loader.load()  # list[Document]

@traceable(name="split_documents", project_name="RAG_Chatbot")
def split_documents(docs, chunk_size=1000, chunk_overlap=150):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    return splitter.split_documents(docs)

@traceable(name="build_vectorstore", project_name="RAG_Chatbot")
def build_vectorstore(splits):
    emb = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
    vs = FAISS.from_documents(splits, emb)
    return vs

@traceable(name="setup_pipeline", project_name="RAG_Chatbot")
def setup_pipeline(pdf_path: str):
    docs = load_pdf(PDF_PATH)
    splits = split_documents(docs)
    vs = build_vectorstore(splits)
    return vs

# ---------- pipeline ----------
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer ONLY from the provided context. If not found, say you don't know."),
    ("human", "Question: {question}\n\nContext:\n{context}")
])

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

# Build the index
vectorstore = setup_pipeline(PDF_PATH)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

parallel = RunnableParallel({
    "context": retriever | RunnableLambda(format_docs),
    "question": RunnablePassthrough(),
})

chain = parallel | prompt | llm | StrOutputParser()

# ---------- run a query ----------
print("PDF RAG ready. Ask a question (or Ctrl+C to exit).")
q = input("\nQ: ").strip()

ans = chain.invoke(q, config=custom_configurations)
print("\nA:", ans)

PDF RAG ready. Ask a question (or Ctrl+C to exit).

Q: what is linear regression?

A: Linear regression is a straightforward approach for predicting a quantitative response on the basis of a single predictor variable. It assumes that there is approximately a linear relationship between the predictor variable and the response.
